In [113]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os
from dotenv import load_dotenv

In [22]:
def convertion_to_table(mans_womens, interes_and_category):
    text = mans_womens + interes_and_category
    dirty_data = pd.DataFrame({text.split('\n')[0]:text.split('\n')[1:]}) #Создаем датафрейм из поступающего текста
    
    dirty_data = pd.concat([dirty_data.loc[3:6], dirty_data.loc[27:46], dirty_data.loc[48:67]]) #Удаляем ненужные строки (они фиксированы)

    #Разделяем таблицу на проценты и категории
    percent = dirty_data[dirty_data[dirty_data.columns[0]].str.contains('%')]
    
    
    category = dirty_data[~dirty_data[dirty_data.colubmns[0]].str.contains('%')]
    category.rename(columns={category.columns[0]: ''}, inplace=True)

    #Создаем очищенную таблицу
    clear_data = percent.set_index(category[''])
    clear_data = clear_data.applymap(lambda x: re.sub(r'\s+', ' ', x).replace(' ',''))
    
    return clear_data.T.reset_index(names='')

In [122]:
def read_login_password(login='login', password='password'):
    load_dotenv()
    return os.getenv(login), os.getenv(password)


# Работа с сайтом

In [92]:
driver = webdriver.Chrome()

In [93]:
driver.get("https://audience.yandex.ru")
button = driver.find_element(By.CLASS_NAME, 'button2__text') #Поиск кнопки ВОЙТИ


In [94]:
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button).click(button).perform() #Нажатие на кнопку ВОЙТИ

In [126]:
login, password = read_login_password('login','password') #Читаем логин и пароль из скрытого файла

In [95]:
login_element = driver.find_element(By.CSS_SELECTOR, "input[placeholder='Логин или email']") # Находим поле ввода Логина

"""Через команду clear() не удалось очистить поле, т.к. присваивается некое значение value
    и счетчик обновить не получается. Пока использую костыль с ctrl+A и Backspace"""
login_element.send_keys(Keys.CONTROL,'A')
login_element.send_keys(Keys.BACK_SPACE)

login_element.send_keys(login)

In [96]:
"""Нажимаем снова на кнопку войти, что бы перейти к вводу пароля"""
button_2 = driver.find_element(By.CSS_SELECTOR, "button[type='submit']") #Поиск кнопки ВОЙТИ
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button_2).click(button_2).perform() #Нажатие на кнопку ВОЙТИ

In [97]:
"""Вводим пароль"""
password_element = driver.find_element(By.CSS_SELECTOR, "input[name='passwd']") # Находим поле ввода Логина

password_element.send_keys(Keys.CONTROL,'A')
password_element.send_keys(Keys.BACK_SPACE)

password_element.send_keys(password)

In [98]:
"""Нажимаем снова на кнопку войти, что бы наконец войти"""
button_3 = driver.find_element(By.CSS_SELECTOR, "button[type='submit']") #Поиск кнопки ВОЙТИ
driver.implicitly_wait(10)
ActionChains(driver).move_to_element(button_3).click(button_3).perform() #Нажатие на кнопку ВОЙТИ

In [102]:
#### """Ищем список из кнопок со статистиками"""
statistick_button = driver.find_elements(
    By.CSS_SELECTOR, 
    "span[class='icon__bars audience-segment-row__stats-button audience-segment-row__stats-button_type_uploading audience-segment-row__stats-button_hint_no icon']"
    )

len(statistick_button)

87

In [110]:
"""Цикл, который будет проходиться по каждой кнопке, внутри делать скриншоты и сохранять их"""
def parcing(first_button=0, last_button=None):

    slicer_statistick_button = statistick_button[first_button:last_button]
    
    for stat_button in slicer_statistick_button:
        stat_button.click() #открытие окна со статистикой
        
        window = driver.find_element(
            By.CSS_SELECTOR, 
            "div[class='audience-segment-statistics audience-segment-statistics_state_ready error-message error-message_type_above i-bem audience-segment-statistics_js_inited']"
            ) #Поиск всего открывшегося окна
        
        window.screenshot(f'screenshots_mans_womens//{window.text.split('\n')[0]}.png') #Сохранение скриншота окна в папку
        mans_womans_stat = window.text #Парсим данные по статистике женщин мужчин с окна
        
        interess_and_category_button = driver.find_element(
            By.ID, 
            "stat-tabs-tab-2"
            ) #Поиск кнопки Интересы и категории
        
        interess_and_category_button.click()
        
        window = driver.find_element(
            By.CSS_SELECTOR, 
            "div[class='audience-segment-statistics audience-segment-statistics_state_ready error-message error-message_type_above i-bem audience-segment-statistics_js_inited']"
                ) #Поиск окна (хз почему старое не подходит, но надо заново найти окно)

        window.screenshot(f'screenshots_interes//{window.text.split('\n')[0]}.png') #Сохранение скриншота окна в папку
        interes_and_category_text = window.text
        
        if stat_button == slicer_statistick_button[0]:
            table = convertion_to_table(mans_womans_stat, interes_and_category_text)
        else:
            merged_table = convertion_to_table(mans_womans_stat, interes_and_category_text)
            table = table.merge(merged_table, how='outer')
    
        #Закрытие окна после всех действий
        driver.find_elements(
            By.CSS_SELECTOR, 
            "span[class='icon__close icon__close_new_yes audience-segment-statistics__close icon']")[0].click()
    return table
        
table = parcing(0,15)

C:\Users\sergey.potashov\AppData\Local\Temp\ipykernel_22268\4171547237.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category.rename(columns={category.columns[0]: ''}, inplace=True)
C:\Users\sergey.potashov\AppData\Local\Temp\ipykernel_22268\4171547237.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clear_data = clear_data.applymap(lambda x: re.sub(r'\s+', ' ', x).replace(' ',''))
C:\Users\sergey.potashov\AppData\Local\Temp\ipykernel_22268\4171547237.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category.rename(columns={category.columns[0]: ''}, inplace=True)
C:\

In [106]:
table.set_index(table.columns[0])

,мужчины,женщины,Авто,Спорт,Финансы,Недвижимость,Туризм,Домашние животные,Семья и дети,Кулинария,...,Автомобилисты,Путешествуют по миру,Путешествуют по стране,Покупают в интернете,Занимаются спортом,Домохозяйки,Следят за модой,Есть дети,Геймеры,Студенты
,,,,,,,,,,,,,,,,,,,,,
Сегмент «GIBDD»,"69,3%","30,7%",189%,140%,151%,162%,134%,130%,119%,114%,...,267%,217%,180%,222%,231%,191%,185%,147%,103%,75%
"Сегмент «MAC_2 км МКАД, 2 ТК Шоколад»","77,9%","22,1%",232%,120%,140%,145%,128%,104%,95%,95%,...,297%,232%,200%,185%,193%,149%,145%,135%,107%,65%
Сегмент «MAC_40-й км трассы А-105 отель ИБИС»,"79,4%","20,6%",211%,119%,148%,141%,138%,93%,82%,83%,...,302%,331%,194%,159%,172%,122%,120%,128%,101%,46%
Сегмент «MAC_Hongqi_17.08.2023-30.04.2024.hashed»,"73,7%","26,3%",193%,147%,119%,123%,119%,128%,121%,89%,...,264%,227%,182%,214%,214%,165%,157%,121%,72%,62%
Сегмент «MAC_MAC_postoyanka-dlya-arendatorru_2»,"63,7%","36,3%",178%,158%,132%,137%,130%,149%,136%,113%,...,218%,190%,171%,214%,203%,176%,161%,130%,76%,79%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Сегмент «г. Пермь, ул. Революции, 13»",49%,51%,140%,171%,138%,148%,142%,110%,127%,108%,...,170%,161%,169%,202%,185%,158%,136%,122%,73%,141%
"Сегмент «г. Самара, Московское шоссе, лит.Д, 28 Б»",56%,44%,151%,151%,130%,137%,129%,103%,118%,101%,...,206%,187%,149%,182%,207%,167%,131%,114%,73%,143%
"Сегмент «г. Тюмень, ул. Республики, д.61»","49,6%","50,4%",150%,154%,142%,179%,136%,108%,119%,107%,...,155%,192%,170%,192%,151%,138%,130%,111%,75%,113%


In [91]:
driver.quit()